In [7]:
from binance.cm_futures import CMFutures
from binance.um_futures import UMFutures
from binance.spot import Spot
from pandas import DataFrame

spot_client = Spot()
cm_futures_client = CMFutures() # settled in cryptocurrency
um_futures_client = UMFutures() # settled in USDT USDC


In [8]:
# get server time
spot_prices = spot_client.ticker_price()
# create df
spot_prices_df = DataFrame(spot_prices)
spot_prices_df

,symbol,price
0,ETHBTC,0.04290000
1,LTCBTC,0.00100400
2,BNBBTC,0.00882100
3,NEOBTC,0.00016720
4,QTUMETH,0.00095600
...,...,...
2705,DOGEJPY,15.78000000
2706,MATICJPY,74.82000000
2707,NEARJPY,705.90000000
2708,TONUSDC,5.66400000


In [9]:
cm_futures_prices = cm_futures_client.ticker_price()
cm_futures_prices_df = DataFrame(cm_futures_prices)
cm_futures_prices_df
# filter all symbols that end with number
cm_futures_prices_df_n = cm_futures_prices_df[cm_futures_prices_df.symbol.str.contains(r'\d$')]
cm_futures_prices_df_n['symbol'].values
# split all values by usd and leav ethe first part
cm_futures_prices_df_n['symbol'].str.split('USD').str[0].values

array(['XRP', 'ETH', 'SOL', 'BNB', 'LINK', 'LTC', 'LTC', 'DOT', 'SOL',
       'BTC', 'LINK', 'ADA', 'DOT', 'BCH', 'BCH', 'BTC', 'ETH', 'XRP',
       'ADA', 'BNB'], dtype=object)

In [10]:
# um_futures_prices = um_futures_client.ticker_price()
# um_futures_prices_df = DataFrame(um_futures_prices)
# um_futures_prices_df

In [11]:
def cash_and_carry(spot_price, futures_price, interest_rate, days_to_expiry):
    return (futures_price - spot_price) / spot_price * 365 / days_to_expiry - interest_rate

def future_premium(cm_futures_prices_df, spot_prices_df, symbol):
    
    pass
# from cm_futures_prices_df get all tickers containing "BTC"
btc_cm_futures_prices_df = cm_futures_prices_df[cm_futures_prices_df['symbol'].str.contains("BTC")]
# print the row ith the highest price
btc_cm_futures_prices_df_max = btc_cm_futures_prices_df[btc_cm_futures_prices_df['price'] == btc_cm_futures_prices_df['price'].max()]
print(btc_cm_futures_prices_df_max)
print(btc_cm_futures_prices_df_max['symbol'].values[0])
max_price = float(btc_cm_futures_prices_df['price'].max())
btc_spot_prices_df = spot_prices_df[spot_prices_df['symbol'] == "BTCUSDT"]
print(btc_spot_prices_df)
spot_price = float(btc_spot_prices_df['price'].values[0])
# Get the difference between the max price and the spot price percentage
percentage = ((max_price - spot_price) / spot_price) * 100
print(percentage)

           symbol      ps    price           time
39  BTCUSD_241227  BTCUSD  65696.7  1724653814783
BTCUSD_241227
     symbol           price
11  BTCUSDT  63837.00000000
2.9132008083086567


In [12]:
def future_premium(cm_futures_prices_df, spot_prices_df, symbol):
    # from cm_futures_prices_df get all tickers containing "BTC"
    btc_cm_futures_prices_df = cm_futures_prices_df[cm_futures_prices_df['symbol'].str.contains(
        symbol)]
    # print the row ith the highest price
    btc_cm_futures_prices_df_max = btc_cm_futures_prices_df[
        btc_cm_futures_prices_df['price'] == btc_cm_futures_prices_df['price'].max()]
    max_price = float(btc_cm_futures_prices_df['price'].max())
    btc_spot_prices_df = spot_prices_df[spot_prices_df['symbol'] == f"{
        symbol}USDT"]
    spot_price = float(btc_spot_prices_df['price'].values[0])
    # Get the difference between the max price and the spot price percentage
    percentage = (((max_price - spot_price) / spot_price) * 100) # - 0.4 # 0.4 is the tx fee
    # rount to 2 decimal places
    return btc_cm_futures_prices_df_max['symbol'].values[0], round(percentage, 2)


cm_futures_prices = cm_futures_client.ticker_price()
cm_futures_prices_df = DataFrame(cm_futures_prices)
cm_futures_prices_df
# filter all symbols that end with number
cm_futures_prices_df_n = cm_futures_prices_df[cm_futures_prices_df.symbol.str.contains(
    r'\d$')]
cm_futures_prices_df_n['symbol'].values
# split all values by usd and leave the first part
symbols = cm_futures_prices_df_n['symbol'].str.split('USD').str[0].values

# create empty dataframe
df_ = DataFrame(columns=['symbol', 'percentage'])
for symbol in symbols:
    s, p = future_premium(cm_futures_prices_df, spot_prices_df, symbol)
    # append to the dataframe
    df_ = df_._append({'symbol': s, 'percentage': p}, ignore_index=True)
    
# sort dataframe by percentage descending
df_ = df_.sort_values(by='percentage', ascending=False)
# remove duplicates
df_ = df_.drop_duplicates()
# reset index
df_ = df_.reset_index(drop=True)
df_

/var/folders/5y/t5w21c695232j0yt3hlpscpm0000gn/T/ipykernel_2496/2438039200.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_ = df_._append({'symbol': s, 'percentage': p}, ignore_index=True)


,symbol,percentage
0,XRPUSD_241227,2.92
1,BTCUSD_241227,2.91
2,LTCUSD_241227,2.87
3,LINKUSD_241227,2.86
4,ETHUSD_241227,2.72
5,SOLUSD_241227,2.48
6,ADAUSD_241227,2.37
7,DOTUSD_240927,0.13
8,BNBUSD_PERP,-0.01
9,BCHUSD_PERP,-0.08
